## JSSP Tabu Search and Genetic Algorithm Example

This notebook demonstraits how to use both prallel tabu seach and the genetic algorithm.

In [1]:
from JSSP.data import Data
import os

# initialize data
data_directory = '../data/given_data'
Data.initialize_data_from_csv(data_directory + '/sequenceDependencyMatrix.csv',
                              data_directory + '/machineRunSpeed.csv',
                              data_directory + '/jobTasks.csv')

# uncomment this to print the data that was read in
# Data.print_data()

### Tabu Search Before Genetic Algorithm

In [3]:
from JSSP.solver import Solver

output_dir = './example_output'
output_results = True
auto_open = True
verbose = False
benchmark = True
progress_bar = True

# ts parameters
runtime = 150 # in seconds
num_processes = 10
tabu_list_size = 50
neighborhood_size = 400
neighborhood_wait = 0.12
probability_change_machine = 0.8
reset_threshold = 100

# ga parameters
runtime = 150 # in seconds
mutation_probability = 0.8
population_size = 120
selection_size = 6
population = []

# create solver
solver = Solver()

# run tabu seach
solution = solver.tabu_search(runtime,
                              num_processes,
                              tabu_list_size,
                              neighborhood_size,
                              neighborhood_wait,
                              probability_change_machine,
                              reset_threshold,
                              benchmark=benchmark,
                              verbose=verbose,
                              progress_bar=progress_bar
                              )
    
print('Tabu Search Solution:')
solution.pprint()

# add all tabu search solutions to population
population += solver.ts_all_solutions

# run genetic algorithm
solution = solver.genetic_algorithm(runtime, 
                                    population, 
                                    population_size, 
                                    mutation_probability, 
                                    selection_size, 
                                    benchmark=benchmark,
                                    verbose=verbose,
                                    progress_bar=progress_bar
                                    )

print('Genetic Algorithm Solution:')
solution.pprint()

if output_results:
    if benchmark:
        solver.output_benchmark_results(output_dir, name='example_benchmark', auto_open=auto_open)
    else:
        solution.create_schedule(output_dir)

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:02:29


Tabu Search Solution:
makespan = 4060.913793103448
machine_makespans = [4058.040650406504, 4060.913793103448, 4058.3552631578955, 4008.734939759036, 4044.8918918918916, 4015.269999999999, 4021.3163265306125, 4043.543046357617]
operation_list =
[[   21     0     0     5 19015]
 [   43     0     0     7  7688]
 [   28     1     0     5  9295]
 ...
 [   31     6     5     2 23583]
 [   44     5     4     0 10137]
 [   47     2     2     4 12399]]


||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:02:27


Genetic Algorithm Solution:
makespan = 4060.913793103448
machine_makespans = [4058.040650406504, 4060.913793103448, 4058.3552631578955, 4008.734939759036, 4044.8918918918916, 4015.269999999999, 4021.3163265306125, 4043.543046357617]
operation_list =
[[   21     0     0     5 19015]
 [   43     0     0     7  7688]
 [   28     1     0     5  9295]
 ...
 [   31     6     5     2 23583]
 [   44     5     4     0 10137]
 [   47     2     2     4 12399]]
opening file:///home/mcfadd/Job_Shop_Schedule_Problem/examples/example_output/example_benchmark in browser


### Genetic Algorithm Before Tabu Search

In [5]:
from JSSP.solver import Solver

output_dir = './example_output'
output_results = False
auto_open = False
verbose = False
benchmark = False
progress_bar = False

# ts parameters
runtime = 10 # in seconds
num_processes = 2
tabu_list_size = 50
neighborhood_size = 250
neighborhood_wait = 0.1
probability_change_machine = 0.8
reset_threshold = 100

# ga parameters
runtime = 10 # in seconds
mutation_probability = 0.8
population_size = 200
selection_size = 5
population = []

# create solver
solver = Solver()

# run geneitc algorithm
solution = solver.genetic_algorithm(runtime, 
                                    population, 
                                    population_size, 
                                    mutation_probability, 
                                    selection_size, 
                                    benchmark=benchmark, 
                                    progress_bar=progress_bar
                                    )

print('Genetic Algorithm Solution:')
solution.pprint()

# pass best solutions found to tabu search
initial_solutions = sorted(solver.ga_result_population)[:num_processes]

# run tabu seach
solution = solver.tabu_search(runtime,
                              num_processes,
                              tabu_list_size,
                              neighborhood_size,
                              neighborhood_wait,
                              probability_change_machine,
                              reset_threshold,
                              initial_solutions=initial_solutions,
                              benchmark=benchmark,
                              verbose=verbose,
                              progress_bar=progress_bar
                              )
    
print('Tabu Search Solution:')
solution.pprint()

if output_results:
    if benchmark:
        solver.output_benchmark_results(output_dir, name='example_benchmark', auto_open=auto_open)
    else:
        solution.create_schedule(output_dir)

Genetic Algorithm Solution:
makespan = 4953.255102040816
machine_makespans = [4784.310270449644, 4931.424700914849, 4875.052631578947, 4696.192771084337, 4016.5405405405413, 4921.30861788618, 4953.255102040816, 4171.747871333964]
operation_list =
[[   43     0     0     5  7688]
 [   13     0     0     4 17404]
 [   19     0     0     2 23693]
 ...
 [   34     4     2     6 18747]
 [   36     3     2     1  9829]
 [   42     2     2     3 17151]]
Tabu Search Solution:
makespan = 4069.3157894736846
machine_makespans = [4060.634146341464, 4030.3634764250532, 4069.3157894736846, 4064.144578313253, 4061.3471182025405, 4064.4199999999996, 4052.1938775510207, 4045.8874172185433]
operation_list =
[[   14     0     0     6 14120]
 [   12     0     0     7  8191]
 [    4     1     0     4 14753]
 ...
 [   44     5     4     5 10137]
 [   45     2     0     7 12334]
 [   42     2     2     3 17151]]
